In [1]:

import pandas as pd
import numpy as np
import hjson

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
import re

# nltk.download('stopwords')
# nltk.download('wordnet')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
from get_dataframes import GetDataframes

from scipy.sparse import hstack

pd.set_option('display.max_columns', None)

In [16]:
with open("config.hjson") as fp:
    config = hjson.load(fp)
datas = GetDataframes(config)

df = datas.get_machine_learning_dataframe(
    "machine_learning",
    # cleaned=True
)

2023-11-10 08:53:12 INFO     Parquet loaded ! Importing machine_learning...
2023-11-10 08:53:12 INFO     Dataframe machine_learning ready to use!


In [4]:
def full_lower(
    text: str
):
    return text.replace(" ", "").replace("-", "")

tt = (
    ("actors", "actors"),
    ("titre_genres", "titre_genres"),
    ("directors", "directors"),
)
for t in tt:
    df[t[0]] = df[t[1]].apply(full_lower)


# df["overview"] = df["overview"].apply(lambda x : x.replace(" ", ","))


In [17]:
df

,titre_id,titre_str,titre_genres,rating_avg,rating_votes,actors,directors,overview
0,tt0035423,Kate & Leopold,"Comedy, Fantasy, Romance",6.40,87897,"Meg Ryan, Hugh Jackman, Liev Schreiber, Brecki...",James Mangold,scientist ex boyfriend discovers portal travel...
1,tt0052832,The Fugitive Kind,"Drama, Romance",7.10,6914,"Anna Magnani, Marlon Brando, Joanne Woodward, ...",Sidney Lumet,val xavier drifter obscure origin arrives smal...
2,tt0052997,The League of Gentlemen,"Comedy, Crime, Drama",7.20,5105,"Jack Hawkins, Roger Livesey, Nigel Patrick, Ri...",Basil Dearden,involuntarily retired colonel hyde recruit sev...
3,tt0053459,Eyes Without a Face,"Drama, Horror",7.60,34278,"Pierre Brasseur, Alexandre Rignault, Alida Val...",Georges Franju,dr g nessier riddled guilt accident caused dis...
4,tt0053579,Late Autumn,"Comedy, Drama",7.90,5854,"Setsuko Hara, Keiji Sada, Mariko Okada, Yôko T...",Yasujirô Ozu,woman daughter forced contend increasing press...
...,...,...,...,...,...,...,...,...
7651,tt9873892,They Cloned Tyrone,"Comedy, Mystery, Sci-Fi",6.60,33197,"Kiefer Sutherland, Jamie Foxx, John Boyega, Te...",Juel Taylor,series eerie event thrust unlikely trio onto t...
7652,tt9883996,Dream Horse,"Biography, Comedy, Drama",6.90,6589,"Alan David, Lynda Baron, Owen Teale, Toni Coll...",Euros Lyn,inspiring true story dream alliance unlikely r...
7653,tt9893250,I Care a Lot,"Comedy, Crime, Drama",6.40,141372,"Dianne Wiest, Rosamund Pike, Peter Dinklage, E...",J Blakeson,court appointed legal guardian defrauds older ...
7654,tt9900782,Kaithi,"Action, Adventure, Crime",8.50,39546,"Narain, Karthi, George Maryan, Arjun Das",Lokesh Kanagaraj,dilli convicted criminal parole meet daughter ...


In [5]:
# feat = [
#     'titre_str',
#     'titre_genres',
#     # 'rating_avg',
#     # 'rating_votes',
#     "actors",
#     'directors',
# ]
# # df.columns.to_list()
# feat

# for f in feat:
#     df[f] = df[f].replace(" ", "")

In [18]:
def title_index(titre):
    return df[df.titre_str == titre].index[0]

def idx_title(idx):
    return df[df.index == idx]["titre_str"].values[0]

In [19]:
def combine(r):
    return r["titre_genres"]+" "+r["directors"]+" "+r["overview"]+" "+r["actors"] #r["titre_str"]+" "+

df["one_for_all"] = df.apply(combine, axis=1)

In [20]:
def choice(movie: str):
    if movie == "genres":
        poids_ = {
            # "titre_str":    0.2,
            "titre_genres": 0.2,
            "actors":       0.2,
            "directors":    0.2,
            "overview":     0.2,
        }


In [21]:
def titre_index(titre: str):
    return df[df.titre_str == titre].index[0]

def director_index(director: str):
    return df[df.directors.str.contains(director)].index[0]

def actor_index(actor: str):
    return df[df.actors.str.contains(actor)].index[0]

def idx_titre(idx: int):
    return df[df.index == idx]["titre_str"].values[0]

def idx_actor(idx: int):
    return df[df.index == idx]["actors"].values[0]

def idx_titre_id(idx: int):
    return df[df.index == idx]["titre_id"].values[0]

# def idx_title(idx):
#     return df[df.index == idx]["titre_str"].values[0]

In [22]:
# tidf = TfidfVectorizer()
# # combined_matrix = tidf.fit_transform(
# #     df["titre_str"]+" "+df["actors"]+" "+df["titre_genres"]+" "+df["directors"]+" "+df["overview"]
# # )
# combined_matrix = tidf.fit_transform(
#     df["one_for_all"]+" "+df["titre_str"]
# )
poids_ = {
    # "titre_str":    0.2,
    "titre_genres": 2,
    "actors":       1.5,
    "directors":    1.5,
    "overview":     2.5,
    # "rating_avg":   0.2,
    # "rating_votes": 0.2,
}

top = 5

full_matrix = []
for col, poids in poids_.items():
    tfidf_ = TfidfVectorizer()
    matrix_ = tfidf_.fit_transform(df[col]) * poids
    full_matrix.append(matrix_)
combined_matrix = hstack(full_matrix)
cosine = cosine_similarity(combined_matrix)
movies = "The Avengers"
mov_idx = titre_index(movies)
similar = cosine[mov_idx]
similar1 = list(enumerate(cosine[mov_idx]))
sim_scores = sorted(similar1, key=lambda x: x[1], reverse=True)
sim_mov_idx = similar.argsort()[::-1][1:top+1]
same_movies = df.loc[sim_mov_idx, "titre_str"]
ttconst = df.loc[sim_mov_idx, "titre_id"]
sim_scores[1:top+1]
score = [i[1] for i in sim_scores]
for movies, tt,  score in zip(same_movies, ttconst, score):
    print(f"Movie : {movies} | id : {tt} | score : {np.round(score, 4)}")


Movie : Still Life | id : tt2395417 | score : 1.0
Movie : Serenity | id : tt0379786 | score : 0.541
Movie : The Walk | id : tt3488710 | score : 0.4076
Movie : Like Someone in Love | id : tt1843287 | score : 0.3842
Movie : Chronicle | id : tt1706593 | score : 0.3595


In [23]:
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
count_matrix = cv.fit_transform(df['one_for_all'])
movie_user_likes = "Fight Club"
movie_index = titre_index(movie_user_likes)

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(count_matrix)
dist, indices = knn_model.kneighbors(count_matrix[movie_index], n_neighbors=15)

print(f"Top 10 similar movies to {movie_user_likes} are :\n")
for index, dis in zip(indices.flatten()[1:], dist.flatten()[1:]):
    print(f"Movie : {idx_title(index)} | id : {idx_titre_id(index)} | score : {np.round(dis, 4)}")


Top 10 similar movies to Fight Club are :

Movie : The Curious Case of Benjamin Button | id : tt0421715 | score : 0.8583
Movie : An Officer and a Spy | id : tt2398149 | score : 0.8588
Movie : Breakfast at Tiffany's | id : tt0054698 | score : 0.8696
Movie : On the Basis of Sex | id : tt4669788 | score : 0.8696
Movie : Kafka | id : tt0102181 | score : 0.8728
Movie : It's Kind of a Funny Story | id : tt0804497 | score : 0.8766
Movie : The Naked Kiss | id : tt0058390 | score : 0.8766
Movie : Snow Angels | id : tt0453548 | score : 0.8769
Movie : Racing with the Moon | id : tt0087968 | score : 0.8784
Movie : May God Save Us | id : tt4944596 | score : 0.8805
Movie : Furioza | id : tt10515864 | score : 0.8815
Movie : Bilal: A New Breed of Hero | id : tt3576728 | score : 0.8834
Movie : The Painted Veil | id : tt0446755 | score : 0.8834
Movie : Space Sweepers | id : tt12838766 | score : 0.8834


In [24]:

# random_model = RandomForestClassifier().fit(combined_matrix, df["titre_genres"])
# for idx, movies in zip(sim_mov_idx, same_movies):
#     mo = combined_matrix[idx].toarray()
#     prediction = random_model.predict(mo)[0]
#     confidence = np.max(random_model.predict_proba(mo))

#     print("movie :", movies)
#     print("confidence :", confidence)

In [25]:
# Movie : Avengers: Age of Ultron | id : tt2395427 | score : 1.0
# Movie : The Gray Man | id : tt1649418 | score : 0.2522
# Movie : Avengers: Endgame | id : tt4154796 | score : 0.1226
# Movie : Snowpiercer | id : tt1706620 | score : 0.1209
# Movie : Kingsman: The Secret Service | id : tt2802144 | score : 0.1117
# print(f"Top 10 similar movies to {movie_user_likes} are :\n")
#

In [26]:
# poids_ = {
#     "titre_str": 0.1,
#     "titre_genres": 0.1,
#     "directors": 0.2,
#     "overview": 0.2,
#     "actor_1": 0.15,
#     "actor_2": 0.1,
    # "actor_3": 0.025,
#     "actor_4": 0.025,
# }

# full_matrix = []
# for col, poids in poids_.items():
#     tfidf_ = TfidfVectorizer()
#     matrix_ = tfidf_.fit_transform(df[col]) * poids
#     full_matrix.append(matrix_)

# combined_matrix = hstack(full_matrix)

# cosine = cosine_similarity(combined_matrix)

# movies = movie
# mov_idx = title_index(movies)
# similar = cosine[mov_idx]
# similar1 = list(enumerate(cosine[mov_idx]))
# sim_scores = sorted(similar1, key=lambda x: x[1], reverse=True)

# sim_mov_idx = similar.argsort()[::-1][1:11]
# same_movies = df.loc[sim_mov_idx, "titre_str"]
# sim_scores[1:11]
# score = [i[1] for i in sim_scores]
# for movies, score in zip(same_movies, score):
#     print(movies, score)


# # random_model = RandomForestClassifier().fit(matrix, df["titre_genres"])
# # for idx, movies in zip(sim_mov_idx, same_movies):
# #     mo = matrix[idx].toarray()
# #     prediction = random_model.predict(mo)[0]
# #     confidence = np.max(random_model.predict_proba(mo))

# #     print("movie :", movies)
# #     print("confidence :", confidence)